# Vulnerability Ireland

TO DO:
- change census names to 10 characters
- allow the domains and dimensions to be weighted differently

## Script Setup

In [1]:
#load the libraries
library(sf)
library(rgdal)
library(rgeos)
library(raster)
library(tidyverse)
library(RColorBrewer)

Linking to GEOS 3.12.1, GDAL 3.8.4, PROJ 9.3.1; sf_use_s2() is TRUE

Loading required package: sp

Please note that rgdal will be retired during October 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See https://r-spatial.org/r/2023/05/15/evolution4.html and https://github.com/r-spatial/evolution
rgdal: version: 1.6-7, (SVN revision 1203)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.8.4, released 2024/02/08
Path to GDAL shared files: /Users/declan/anaconda3/envs/ISVEHI/share/gdal
 GDAL does not use iconv for recoding strings.
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 9.3.1, December 1st, 2023, [PJ_VERSION: 931]
Path to PROJ shared files: /Users/declan/Library/Application Support/proj:/Users/declan/anaconda3/envs/ISVEHI/share/proj
PROJ CDN enabled: TRUE
Linking to sp version:2.1-2
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rg

In [2]:
#set a directory for export 
exportDirectory <- "../2_OutputData"

# Get the data

In [3]:
#census data
censusData <- read.csv("../1_InputData/1a_CensusData/censusDataZ.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
# head(censusData)

#impervious surface
imperviousSurfaceData <- read.csv("../1_InputData/1c_ImperviousSurface/imperviousSurface.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
imperviousSurfaceData <- imperviousSurfaceData %>% select('SA_GUID__1', 'impervious')
# head(imperviousSurfaceData)

#vulnerabiltiy combine info to help guide the amalgamation of the data - weighting should be changed in this file.
combineInfo <- read.csv("../0_Scripts/VulnerabilityIndicators/vulnerabilityIndicatorsInformation.csv", header=TRUE, sep=",", stringsAsFactors = FALSE, fileEncoding="UTF-8-BOM")
#replace spaces in column names with _
names(combineInfo) <- gsub('\\.', '_', names(combineInfo))
combineInfo

domain,indicator,sens,prepare,respond,recover,enhExp,indVulRel,weighting
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
age,young,1,0,0,0,0,1,0.5000000
age,old,1,0,0,0,0,1,0.5000000
health,poorHealth,1,0,0,0,0,1,0.5000000
health,disability,1,0,0,0,0,1,0.5000000
income,unemploy,0,1,1,1,0,1,0.2000000
income,lowSkill,0,1,1,1,0,1,0.2000000
income,farming,0,1,1,1,0,1,0.2000000
income,oneParent,0,1,1,1,0,1,0.2000000
income,familyUnits,0,1,1,1,0,1,0.2000000


## Combine the data

In [4]:
#combine the data into one master dataset
masterDataset <- merge(censusData, imperviousSurfaceData, by = "SA_GUID__1")
head(masterDataset)

,SA_GUID__1,young,old,priSch,notVolunteers,poorHealth,disability,unemploy,lowSkill,farming,⋯,yearBuilt,mobHome,unoccupiedDwellings,oneParent,onePerson,noCar,noInternet,priWater,familyUnits,impervious
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,3.3880435,-0.8683184,1.7138413,0.88627118,-0.72494112,-0.92235833,-0.61650259,-0.3191812,-0.5775435,⋯,-0.8371139,-0.1877926,-0.2345322,-0.9537932,-1.4596182731,-0.80430587,-0.6252523,-0.5390374,-0.28864382,-0.3062535
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-1.2041476,1.0830212,-0.5610249,-0.81754631,0.27801178,-0.62611003,-0.23523246,-0.7175350,0.2814244,⋯,-0.4334056,-0.1877926,-0.3975997,-0.5813707,-0.0009715101,-0.65728630,1.0787065,1.3835981,0.24855343,-0.4046186
3,002b83e4-5cce-414a-8570-871a50ae0964,0.2940109,-0.6563918,0.3585112,0.52365099,0.07647292,0.06773408,0.22240140,-0.7370951,-0.1126775,⋯,-0.7250643,-0.1877926,-0.5907831,0.3083236,-0.2053615213,-0.07351769,0.1779874,-0.5390374,-0.06192013,0.2528440
4,005eec3f-d746-480f-ae2c-bc6f1151548d,-1.3337162,1.4547264,-1.3574765,-0.27505307,0.49313225,0.45603900,-0.07660151,-1.9157382,-0.5775435,⋯,-0.7827532,-0.1877926,-0.3274716,-1.0918818,-0.6525911498,1.07997948,-1.2116091,-0.3106055,-1.18351563,0.5785988
5,00b00ae4-229d-455d-84f1-d6face4876b1,1.7131001,-1.2060316,2.9123134,1.59439615,-0.32805634,-0.03080027,0.75768133,1.5271694,-0.5775435,⋯,-0.7424513,-0.1877926,-0.6509695,3.2416831,-1.1159542610,-0.02238299,-0.8408819,-0.5390374,1.26642096,0.5467843
6,00df2b78-66c8-4009-89e0-0e14e15e4208,1.5084163,-0.8589597,0.6262758,-0.02986993,-1.03118100,-1.02730679,-0.97841257,0.4816479,-0.4599295,⋯,-0.6196710,-0.1877926,-0.4166064,-0.4959203,-0.9415702944,-0.83850016,-0.7636471,-0.5390374,-0.57201990,-0.4271954


# Indicator Correlation

Need to check if the indicators are too strongly correlated with each other (+-0.85).

In [5]:
masterDatasetCor <- masterDataset %>% select(-'SA_GUID__1')
indicatorCorrelation <- round(cor(masterDatasetCor), 2)
indicatorCorrelation

,young,old,priSch,notVolunteers,poorHealth,disability,unemploy,lowSkill,farming,rent,⋯,yearBuilt,mobHome,unoccupiedDwellings,oneParent,onePerson,noCar,noInternet,priWater,familyUnits,impervious
young,1.00,-0.32,0.58,0.11,-0.20,-0.35,-0.13,0.01,-0.02,0.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.09
old,-0.32,1.00,-0.37,-0.21,0.31,0.52,0.08,-0.02,0.11,-0.32,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.15
priSch,0.58,-0.37,1.00,0.01,-0.23,-0.37,-0.14,0.10,0.05,-0.12,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.23
notVolunteers,0.11,-0.21,0.01,1.00,0.25,0.13,0.36,0.21,-0.40,0.50,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.48
poorHealth,-0.20,0.31,-0.23,0.25,1.00,0.64,0.57,0.29,-0.13,0.22,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.16
disability,-0.35,0.52,-0.37,0.13,0.64,1.00,0.55,0.28,-0.12,0.13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.10
unemploy,-0.13,0.08,-0.14,0.36,0.57,0.55,1.00,0.44,-0.10,0.39,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.13
lowSkill,0.01,-0.02,0.10,0.21,0.29,0.28,0.44,1.00,0.07,0.06,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.14
farming,-0.02,0.11,0.05,-0.40,-0.13,-0.12,-0.10,0.07,1.00,-0.45,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.58
rent,0.00,-0.32,-0.12,0.50,0.22,0.13,0.39,0.06,-0.45,1.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.59


# Data Processing

## Indicator Relationship

In [6]:
#get the indicator relationship with vulnerability (+1 or -1)
indRelationship <- combineInfo %>% select('indicator', 'indVulRel')
indRelationship <- indRelationship %>%
    spread(key = 'indicator', value = 'indVulRel')
indRelationship

disability,education,engLang,familyUnits,farming,impervious,lowSkill,mobHome,newRes,noCar,⋯,onePerson,poorHealth,priSch,priWater,rent,travelTime,unemploy,unoccupiedDwellings,yearBuilt,young
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,1,1,1,1,1,1,1,⋯,1,1,-1,1,1,1,1,1,1,1


In [7]:
#multiply the indicators by their vulnerabiltiy relationship

#get the column names and weights
names <- names(indRelationship)
weights <- indRelationship[, names]

#copy and rename the dataset
indicatorRelationshipDataset <- masterDataset

#multiply the data
indicatorRelationshipDataset[, names] <- sweep(indicatorRelationshipDataset[, names], 2, unlist(weights[, names]), "*")
# indicatorRelationshipDataset

## Domains

In [8]:
# get the domains and their associated indicator ID
domainsIndicators <- combineInfo %>% select('domain', 'indicator')
domainsIndicators

domain,indicator
<chr>,<chr>
age,young
age,old
health,poorHealth
health,disability
income,unemploy
income,lowSkill
income,farming
income,oneParent
income,familyUnits


In [9]:
#get a list of the unique domains
domains <- unique(domainsIndicators$domain)
domains

[1] "age"      "health"   "income"   "info"     "locKnow"  "mobility"
 [7] "access"   "socNet"   "tenure"   "physEnv"  "houChar"

In [10]:
#create the start of the dataset in which further data will be added with the GUID
domainDatasetMerge <- masterDataset %>% select('SA_GUID__1')

#loop through each the domains and:
for (domainI in domains){
    #identify which indicators are used within the domain
    domainData <- domainsIndicators %>% filter(domain == domainI)
    
    #get a vector of indicator IDs for the domain
    domainDataIDs <- domainData$indicator
#     print(domainDataIDs)
  
    #count the number of indicators in the domain
    indicatorCount <- length(domainData$indicator)
#     print(indicatorCount)
    
    #add the GUID column name to the vector
    domainDataIDs <- (c('SA_GUID__1', domainDataIDs))
#     str(domainDataIDs)
    
#     filter the master dataset to only the indicators in the domain
    masterDatasetDomainFiltered <- indicatorRelationshipDataset[domainDataIDs]
#    str(masterDatasetDomainFiltered)

    #calculate the weight of each indicator (assumed to be EQUAL)
    weighting <- 1/indicatorCount
#     print(weighting)
          
    #weight the data
    weightedData <- masterDatasetDomainFiltered %>% mutate_if(is.numeric, function(x) {x*weighting})
#     str(weightedData)
    
    
    #sum the data to get the total score for the domain
    weightedData[, domainI] <- rowSums(weightedData[2:(indicatorCount+1)], na.rm = TRUE)
    
    #add the total for the domain to the merging table
    domainDatasetMerge <- merge(domainDatasetMerge, weightedData %>% select('SA_GUID__1', domainI), by = 'SA_GUID__1')
    
}

#all the total domain scores collated into one table
head(domainDatasetMerge)


Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(domainI)

  # Now:
  data %>% select(all_of(domainI))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


,SA_GUID__1,age,health,income,info,locKnow,mobility,access,socNet,tenure,physEnv,houChar
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,1.25986257,-0.8236497,-0.55113286,-0.3552132,0.6516664,-0.80430587,1.0782176,-0.76239614,-0.6291412,-0.3062535,-0.8889340
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.06056320,-0.1740491,-0.20083207,-0.1481675,-0.2804627,-0.65728630,-0.4619480,-0.08583099,-1.0319325,-0.4046186,-0.1002992
3,002b83e4-5cce-414a-8570-871a50ae0964,-0.18119048,0.0721035,-0.07619354,0.3552067,3.1071851,-0.07351769,2.0566182,-0.01340725,0.3733581,0.2528440,-0.5095051
4,005eec3f-d746-480f-ae2c-bc6f1151548d,0.06050512,0.4745856,-0.96905613,-0.5608664,4.2238731,1.07997948,-0.7677528,0.14327742,1.1891164,0.5785988,-0.2665274
5,00b00ae4-229d-455d-84f1-d6face4876b1,0.25353426,-0.1794283,1.24308226,-0.6141542,-0.7106716,-0.02238299,-0.5840908,-0.81129050,2.1279286,0.5467843,-0.9254807
6,00df2b78-66c8-4009-89e0-0e14e15e4208,0.32472828,-1.0292439,-0.40492687,-0.4923798,0.9026253,-0.83850016,3.0768631,-0.53257202,-0.6391705,-0.4271954,-0.6490079


## Dimensions

Need to collate the domains into the five dimensions.

In [11]:
# create a vector of the dimension names
dimensions <- c('sens', 'prepare', 'respond', 'recover', 'enhExp')

#create the dataset to combine the results with just the GUID present
dimensionDatasetMerge <- masterDataset %>% select('SA_GUID__1')

# get the domain and dimension information
dimensionInfo <- combineInfo %>% select(c('domain', dimensions))
# dimensionInfo

# loop through each of the dimensions and:
for (dimensionI in dimensions){
    #get all of the domains and a single dimension
    dimensionIndicators <- dimensionInfo %>% select(c('domain', dimensionI))
    
    #filter the domains by the dimension - 1 means that the domain is part of the dimension
    dimensionIndicators <- dimensionIndicators %>% filter(dimensionIndicators[, dimensionI] == 1)
       
    #get a unique vector of the domains
    domainsDimensionIndicator <- unique(dimensionIndicators$domain)
    
    #get the number of domains that is in the dimension
    domainsDimensionIndicatorCount = length(domainsDimensionIndicator)
#     print(domainsDimensionIndicatorCount)
    
    #create a dataset with only the domains for the dimension
    dimensionTotal <- domainDatasetMerge %>% select(c('SA_GUID__1', domainsDimensionIndicator))  
    str(dimensionTotal)
    
#     #create the new dimension field name
#     dimensionName = paste(dimensionI)
    
    #get the sum of the dimensions
    dimensionTotal[, dimensionI] <- rowSums(dimensionTotal[2:(domainsDimensionIndicatorCount+1)], na.rm = TRUE)
#      str(dimensionTotal)
    
    #add the total for the dimension to the merging table
    dimensionDatasetMerge <- merge(dimensionDatasetMerge, dimensionTotal %>% select('SA_GUID__1', dimensionI), by = 'SA_GUID__1')   
    
}
head(dimensionDatasetMerge)

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(dimensions)

  # Now:
  data %>% select(all_of(dimensions))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”
Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(dimensionI)

  # Now:
  data %>% select(all_of(dimensionI))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”
Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(domainsDimensionIndicator)

  # Now:
  data %>% select(all_of(domainsDimensionIndicator))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


'data.frame':	18919 obs. of  3 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ age       : num  1.2599 -0.0606 -0.1812 0.0605 0.2535 ...
 $ health    : num  -0.8236 -0.174 0.0721 0.4746 -0.1794 ...
'data.frame':	18919 obs. of  5 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ income    : num  -0.5511 -0.2008 -0.0762 -0.9691 1.2431 ...
 $ info      : num  -0.355 -0.148 0.355 -0.561 -0.614 ...
 $ locKnow   : num  0.652 -0.28 3.107 4.224 -0.711 ...
 $ tenure    : num  -0.629 -1.032 0.373 1.189 2.128 ...
'data.frame':	18919 obs. of  7 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2

,SA_GUID__1,sens,prepare,respond,recover,enhExp
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43621285,-0.8838208,-0.7431641,-2.4730481,-1.1951875
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461232,-1.6613948,-1.8345275,-1.0921168,-0.5049177
3,002b83e4-5cce-414a-8570-871a50ae0964,-0.10908698,3.7595564,5.3558916,0.1920882,-0.2566610
4,005eec3f-d746-480f-ae2c-bc6f1151548d,0.53509075,3.8830670,3.1494548,-0.3066656,0.3120713
5,00b00ae4-229d-455d-84f1-d6face4876b1,0.07410596,2.0461851,-1.4995078,-0.2047454,-0.3786964
6,00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70451561,-0.6338519,1.7111095,-2.2683788,-1.0762033


In [12]:
#standardise the data

#copy the data so that Z scores can be calculated
dimensionDatasetMergeZ <- dimensionDatasetMerge

#if the column is a numeric, then scale the data
dimensionDatasetMergeZ %>% mutate_if(is.numeric, scale)

head(dimensionDatasetMergeZ)

SA_GUID__1,sens,prepare,respond,recover,enhExp
<chr>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.38290372,-0.4656315,-0.31341367,-1.22413099,-1.108264158
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.20594059,-0.8752879,-0.77367304,-0.54058556,-0.468196203
002b83e4-5cce-414a-8570-871a50ae0964,-0.09575557,1.9806817,2.25873362,0.09508151,-0.237994657
005eec3f-d746-480f-ae2c-bc6f1151548d,0.46969785,2.0457519,1.32821571,-0.15179603,0.289375093
00b00ae4-229d-455d-84f1-d6face4876b1,0.06504954,1.0780105,-0.63238558,-0.10134666,-0.351154610
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.61841747,-0.3339380,0.72162412,-1.12282202,-0.997933458
0117a71e-3e6a-445e-be5a-b4e79e6feb78,-0.89414520,-0.6879552,0.37635953,-0.92114157,-0.373346359
0138d4f1-7fda-46f9-9ccb-8821c5b9ac4c,-1.43925635,5.9465232,4.55637305,0.06655019,-0.154294568
01f84d32-7411-4d29-bdd3-92c513c11e85,-0.14792397,0.8028242,0.35528603,-0.01452790,-0.552960550


,SA_GUID__1,sens,prepare,respond,recover,enhExp
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43621285,-0.8838208,-0.7431641,-2.4730481,-1.1951875
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461232,-1.6613948,-1.8345275,-1.0921168,-0.5049177
3,002b83e4-5cce-414a-8570-871a50ae0964,-0.10908698,3.7595564,5.3558916,0.1920882,-0.2566610
4,005eec3f-d746-480f-ae2c-bc6f1151548d,0.53509075,3.8830670,3.1494548,-0.3066656,0.3120713
5,00b00ae4-229d-455d-84f1-d6face4876b1,0.07410596,2.0461851,-1.4995078,-0.2047454,-0.3786964
6,00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70451561,-0.6338519,1.7111095,-2.2683788,-1.0762033


## Vulnerability

In [13]:
#sum the dimensions to create a total overall score of vulnerability
dimensionDatasetMergeZ$vuln <- rowSums(dimensionDatasetMergeZ[2:(ncol(dimensionDatasetMergeZ))], na.rm = TRUE)
head(dimensionDatasetMergeZ)

#scale the overall vulnerability
dimensionDatasetMergeZ$vuln <- scale(dimensionDatasetMergeZ$vuln)

#scale makes the data a list(?), so need to make it numeric to allow the shapefile to be created at export
dimensionDatasetMergeZ$vuln<- as.numeric(dimensionDatasetMergeZ$vuln)
head(dimensionDatasetMergeZ)

,SA_GUID__1,sens,prepare,respond,recover,enhExp,vuln
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43621285,-0.8838208,-0.7431641,-2.4730481,-1.1951875,-4.85900760
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461232,-1.6613948,-1.8345275,-1.0921168,-0.5049177,-5.32756914
3,002b83e4-5cce-414a-8570-871a50ae0964,-0.10908698,3.7595564,5.3558916,0.1920882,-0.2566610,8.94178822
4,005eec3f-d746-480f-ae2c-bc6f1151548d,0.53509075,3.8830670,3.1494548,-0.3066656,0.3120713,7.57301824
5,00b00ae4-229d-455d-84f1-d6face4876b1,0.07410596,2.0461851,-1.4995078,-0.2047454,-0.3786964,0.03734159
6,00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70451561,-0.6338519,1.7111095,-2.2683788,-1.0762033,-2.97184015


,SA_GUID__1,sens,prepare,respond,recover,enhExp,vuln
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43621285,-0.8838208,-0.7431641,-2.4730481,-1.1951875,-0.713676211
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461232,-1.6613948,-1.8345275,-1.0921168,-0.5049177,-0.782497101
3,002b83e4-5cce-414a-8570-871a50ae0964,-0.10908698,3.7595564,5.3558916,0.1920882,-0.2566610,1.313342573
4,005eec3f-d746-480f-ae2c-bc6f1151548d,0.53509075,3.8830670,3.1494548,-0.3066656,0.3120713,1.112301814
5,00b00ae4-229d-455d-84f1-d6face4876b1,0.07410596,2.0461851,-1.4995078,-0.2047454,-0.3786964,0.005484619
6,00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70451561,-0.6338519,1.7111095,-2.2683788,-1.0762033,-0.436494814


## Merge

In [14]:
#merge all the indicators, domains, dimensions, and total vulnerability into one dataset
outputDataset <- merge(masterDataset, domainDatasetMerge, by = 'SA_GUID__1')
outputDataset <- merge(outputDataset, dimensionDatasetMergeZ, by = 'SA_GUID__1')
str(outputDataset)

'data.frame':	18919 obs. of  44 variables:
 $ SA_GUID__1             : chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ young                  : num  3.388 -1.204 0.294 -1.334 1.713 ...
 $ old                    : num  -0.868 1.083 -0.656 1.455 -1.206 ...
 $ priSch                 : num  1.714 -0.561 0.359 -1.357 2.912 ...
 $ notVolunteers          : num  0.886 -0.818 0.524 -0.275 1.594 ...
 $ poorHealth             : num  -0.7249 0.278 0.0765 0.4931 -0.3281 ...
 $ disability             : num  -0.9224 -0.6261 0.0677 0.456 -0.0308 ...
 $ unemploy               : num  -0.6165 -0.2352 0.2224 -0.0766 0.7577 ...
 $ lowSkill               : num  -0.319 -0.718 -0.737 -1.916 1.527 ...
 $ farming                : num  -0.578 0.281 -0.113 -0.578 -0.578 ...
 $ rent                   : num  -0.629 -1.032 0.373 1.189 2.128 ...
 $ education              : num  -0.317 -0.699 0.152 -

# Export

In [15]:
#set the export path
exportPath <- paste(exportDirectory, 'irelandVulnerability.csv', sep = '/')

#write the output to a csv
write.csv(outputDataset, exportPath, row.names = FALSE)

#get the small area spatial data
smallAreaShp <- st_read("../0_Scripts/downloads/SMALL_AREA_2022.shp")

#get just the GUID from the dataset
smallAreaShp <- smallAreaShp[c('SA_GUID__1')]

# #add the data to the spatial data using the GUID as the common field
smallAreaVulnerability <- merge(smallAreaShp, outputDataset, by ="SA_GUID__1")

#export to shp and geojson
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.geojson", sep = ""), delete_dsn=TRUE)
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.shp", sep = ""), append = FALSE)

Reading layer `SMALL_AREA_2022' from data source 
  `/Volumes/2023_WD18TB/REACHOUT/git/ISVEHI/0_Scripts/downloads/SMALL_AREA_2022.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 18919 features and 28 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 417471.5 ymin: 519663.7 xmax: 734481.1 ymax: 966896.3
Projected CRS: IRENET95 / Irish Transverse Mercator
Deleting source `../2_OutputData/irelandVulnerability.geojson' using driver `GeoJSON'
Writing layer `irelandVulnerability' to data source 
  `../2_OutputData/irelandVulnerability.geojson' using driver `GeoJSON'
Writing 18919 features with 44 fields and geometry type Multi Polygon.


Warning message in abbreviate_shapefile_names(obj):
“Field names abbreviated for ESRI Shapefile driver”


Deleting layer `irelandVulnerability' using driver `ESRI Shapefile'
Writing layer `irelandVulnerability' to data source 
  `../2_OutputData/irelandVulnerability.shp' using driver `ESRI Shapefile'
Writing 18919 features with 44 fields and geometry type Multi Polygon.


**END**